In [1]:
# 라이브러리 불러오기
from selenium import webdriver
from bs4 import BeautifulSoup
import time

from tqdm.notebook import tqdm # 상태진행바
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import json

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import re
from datetime import datetime

#excel control library, ! pip install openpyxl
from openpyxl import Workbook

import requests

In [2]:
url_list = ['https://m.place.naver.com/restaurant/166240712/home', 'https://m.place.naver.com/restaurant/1083519696/home']

In [3]:
def get_url_list(url):
    blog_list = []
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 크롬 창이 뜨지 않도록 설정
    browser = webdriver.Chrome(service=service)
    browser.get(url)
    desired_text = "리뷰"
    try:
        menu_list = WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.tpj9w._tab-menu')))

        for menu in menu_list:
            if menu.text == desired_text:
                menu.click()  # 원하는 텍스트가 있는 메뉴를 클릭
                break

        review_list = WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.YsfhA')))
        if len(review_list) > 1:
            review_list[1].click()  # Click on the review tab
            
        while True:
            try:
                more_button = WebDriverWait(browser, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.fvwqf')))
                more_button.click()
                time.sleep(0.4)  # Moderate delay to simulate user behavior
            except:
                break
        
        html = browser.page_source
        bs = BeautifulSoup(html, 'lxml')
        blogs = bs.select("li.xg2_q")

        for i in blogs:
            a_tag = i.find('a', href=True)
            if a_tag:
                blog_list.append(a_tag['href'])
    except:
        print("get_url_list_error: " + url)
        return False
    else:
        return blog_list
    finally:
        browser.quit()    

In [4]:
def get_blog_reviews(url_list):
    for idx, url in enumerate(url_list):
        blog_url_list = get_url_list(url)
        df_text = pd.DataFrame(columns=['date', 'url', 'title', 'text']) 
        df_idx = 0
        if blog_url_list:
            for url in blog_url_list:
                res = requests.get(url)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "lxml")
                try:
                    date = soup.find("p", attrs={'class':'blog_date'}).text.rstrip()
                    title = soup.find("span", attrs={'class':'se-fs- se-ff-'}).text
                    content = soup.find("div", attrs={'class':'se-main-container'}).text
                    content = content.replace('\n', '')
                    #content = content.replace('\u200b', '')

                    df_text.loc[df_idx] = [date, url, title, content]
                    df_idx += 1
                    time.sleep(0.5)
                except Exception as e:
#                     print(f"blog error: URL: {url}, Error: {str(e)}")
                    continue
            csv_name = 'data/'+ str(idx) + '.csv'
            df_text.to_csv(csv_name, index=False, encoding='utf-8-sig')
            print("clear: " + str(idx))
        else:
            print("no blog_url_list")
            continue

In [5]:
df = pd.read_csv('store.csv')
desired_columns = df['링크']

In [6]:
# 문자열에 'https://'를 추가합니다.
url_list = ['https://' + url for url in desired_columns]

In [7]:
get_blog_reviews(url_list)

clear: 0
clear: 1
clear: 2
clear: 3
clear: 4
clear: 5
clear: 6
clear: 7
get_url_list_error: https://m.place.naver.com/restaurant/1585175133/home
no blog_url_list
clear: 9
clear: 10
clear: 11
clear: 12
clear: 13
clear: 14
clear: 15
clear: 16
clear: 17
clear: 18
clear: 19
clear: 20
clear: 21
clear: 22
clear: 23
clear: 24
clear: 25
clear: 26
clear: 27
clear: 28
clear: 29
clear: 30
clear: 31
clear: 32
clear: 33
clear: 34
clear: 35
clear: 36
clear: 37
clear: 38
clear: 39
clear: 40
clear: 41
clear: 42
clear: 43
clear: 44
clear: 45
clear: 46
get_url_list_error: https://m.place.naver.com/restaurant/1019293954/home
no blog_url_list
get_url_list_error: https://m.place.naver.com/restaurant/1880578521/home
no blog_url_list
clear: 49
clear: 50
clear: 51
get_url_list_error: https://m.place.naver.com/restaurant/1009817270/home
no blog_url_list
clear: 53
clear: 54
get_url_list_error: https://m.place.naver.com/restaurant/1439850593/home
no blog_url_list
clear: 56
clear: 57
clear: 58
get_url_list_error: